# Plot up wind speed in NEMO forcing file

In [1]:
import xarray as xr
import matplotlib.pyplot as plt
import math
import numpy as np
import arrow
import cmocean

In [2]:
%matplotlib inline

## load data

In [59]:
ds = xr.open_dataset('https://salishsea.eos.ubc.ca/erddap/griddap/ubcSSaSurfaceAtmosphereFieldsV1')
ds

<xarray.Dataset>
Dimensions:    (gridX: 256, gridY: 266, time: 36288)
Coordinates:
  * time       (time) datetime64[ns] 2014-09-12 ... 2018-11-01T23:00:00
  * gridY      (gridY) float64 0.0 2.5e+03 5e+03 ... 6.575e+05 6.6e+05 6.625e+05
  * gridX      (gridX) float64 0.0 2.5e+03 5e+03 ... 6.35e+05 6.375e+05
Data variables:
    atmpres    (time, gridY, gridX) float32 ...
    precip     (time, gridY, gridX) float32 ...
    qair       (time, gridY, gridX) float32 ...
    solar      (time, gridY, gridX) float32 ...
    tair       (time, gridY, gridX) float32 ...
    therm_rad  (time, gridY, gridX) float32 ...
    u_wind     (time, gridY, gridX) float32 ...
    v_wind     (time, gridY, gridX) float32 ...
Attributes:
    acknowledgement:           MEOPAR, ONC, Compute Canada, Environment Canada
    cdm_data_type:             Grid
    Conventions:               CF-1.6, COARDS, ACDD-1.3
    coverage_content_type:     modelResult
    creator_email:             sallen@eos.ubc.ca
    creator_name:

## Calculate wind speed.  

In [60]:
ds.u_wind

<xarray.DataArray 'u_wind' (time: 36288, gridY: 266, gridX: 256)>
[2471067648 values with dtype=float32]
Coordinates:
  * time     (time) datetime64[ns] 2014-09-12 ... 2018-11-01T23:00:00
  * gridY    (gridY) float64 0.0 2.5e+03 5e+03 ... 6.575e+05 6.6e+05 6.625e+05
  * gridX    (gridX) float64 0.0 2.5e+03 5e+03 ... 6.325e+05 6.35e+05 6.375e+05
Attributes:
    _ChunkSizes:  [  1 266 256]
    level:        10 m above ground
    long_name:    U-Component of Wind
    short_name:   UGRD_10maboveground
    units:        m/s

### Figure out how to work with math.sqrt to calculate wind speed for 3D matrix

In [35]:
lt = 0
ly = 0
lx = 0
wind_spd = np.zeros((it,iy,ix))
wind_spd = np.array(math.sqrt(ds.u_wind[it,iy,ix]**2 + ds.v_wind[it,iy,ix]**2))

In [51]:
ds.u_wind.gridX

<xarray.DataArray 'gridX' (gridX: 256)>
array([      0.,    2500.,    5000., ...,  632500.,  635000.,  637500.])
Coordinates:
  * gridX    (gridX) float64 0.0 2.5e+03 5e+03 ... 6.325e+05 6.35e+05 6.375e+05
Attributes:
    _ChunkSizes:    256
    actual_range:   [      0.  637500.]
    long_name:      X
    standard_name:  x

In [52]:
getattr(ds.u_wind.gridX,'_ChunkSizes')

256

In [69]:
lt = getattr(ds.u_wind.time,'_ChunkSizes')
lx = getattr(ds.u_wind.gridX,'_ChunkSizes')
ly = getattr(ds.u_wind.gridY,'_ChunkSizes')

In [64]:
ly

266

In [65]:
lx

256

In [66]:
lt

1

###  Right.  Not what I want for "lt"....  Take two.

In [70]:
ds.u_wind.shape

(36288, 266, 256)

In [71]:
ds.u_wind.shape[1]

266

In [73]:
lt = ds.u_wind.shape[0]
ly = ds.u_wind.shape[1]
lx = ds.u_wind.shape[2]

In [74]:
lt

36288

In [75]:
ly

266

In [76]:
lx

256

###  That's better!!! :-) 

###  OK.  Let's try this AGAIN!  Can I to matrix sqrt or do I need to loop?
###  Firs: Matrix square root

In [80]:
wind_spd = np.zeros((lt,ly,lx))
wind_spd = np.array(math.sqrt(ds.u_wind**2 + ds.v_wind**2))

RuntimeError: NetCDF: DAP server error

In [81]:
wind_spd[0,:,:] = np.array(math.sqrt(ds.u_wind[0,:,:]**2 + ds.v_wind[0,:,:]**2))

TypeError: only length-1 arrays can be converted to Python scalars

In [82]:
wind_spd[0,0,:] = np.array(math.sqrt(ds.u_wind[0,0,:]**2 + ds.v_wind[0,0,:]**2))

TypeError: only length-1 arrays can be converted to Python scalars

In [83]:
wind_spd[0,0,0] = np.array(math.sqrt(ds.u_wind[0,0,0]**2 + ds.v_wind[0,0,0]**2))

### Well...for loops it is then

In [90]:
test = [0,1,2,3]
for ix in test:
    print(test[ix])


0
1
2
3


###  Ah.  Wow.  I just learned that I went through all that effort to figure out the indexing of the matrix for NO GOOD REASON!!!!!

In [97]:
for it in ds.u_wind:
    for iy in ds.u_wind:
        for ix in ds.u_wind:
            wind_spd[it,iy,ix] = np.array(math.sqrt(ds.u_wind[it,iy,ix]**2 + ds.v_wind[it,iy,ix]**2))
            

TypeError: invalid indexer array, does not have integer dtype: array([[-6.14696121, -6.18696117, -6.21696091, ..., -0.37696105,
        -0.44196105, -0.54196107],
       [-6.08696127, -6.12196112, -6.15696096, ..., -0.63196105,
        -0.68196106, -0.86696106],
       [-6.04196119, -6.06196117, -6.09696102, ..., -0.87696105,
        -0.94696105, -1.18196106],
       ..., 
       [-0.22696106, -0.19696106, -0.17196105, ...,  0.18803895,
         0.62803894,  0.84303892],
       [-0.23696107, -0.21196106, -0.19196106, ...,  0.24303894,
         0.49303895,  0.75303894],
       [-0.26196107, -0.25196105, -0.21196106, ...,  0.24303894,
         0.34303895,  0.62303895]], dtype=float32)

### Back to the drawing board!

In [101]:
for it in range(0,3):
        wind_spd[it,0,0] = np.array(math.sqrt(ds.u_wind[it,0,0]**2 + ds.v_wind[it,0,0]**2))
        print(wind_spd[it,0,0])


10.5247632253
11.0023575009
11.7945448264


In [ ]:
for it in range(0,lt-1):
    for iy in range(0,ly-1):
        for ix in range(0,lx-1):
            wind_spd[it,iy,ix] = np.array(math.sqrt(ds.u_wind[it,iy,ix]**2 + ds.v_wind[it,iy,ix]**2))
            